In [1]:
# This version number is appended to each generated file as a suffix
version = 120

In [2]:
# Importing the required libraries
import pandas as pd
import numpy as np

## Loading Data

In [3]:
# Load the dataset after the exploratory data analysis
challenge_set_updated = pd.read_csv("./data/challenge_set_updated_v20.csv")
submission_set_updated = pd.read_csv("./data/submission_set_updated_v20.csv")
submission_set = pd.read_csv("./data/final_submission_set.csv")

## Predictive Model Learning

In [4]:
# Defining the categorical features names
cat_names = ['callsign',
            'adep', 
            'ades', 
            'aircraft_type', 
            'wtc', 
            'airline',
            'offblock_hour',
            'offblock_minute', 
            'offblock_day_of_week',
            'offblock_weekday_name',
            'offblock_month',
            'offblock_week_of_year', 
            'offblock_season', 
            'arrival_hour',
            'arrival_minute',
            'arrival_season',
            'arrival_weekday_name',
            'is_offblock_weekend',
            'is_offblock_rush_hour',
            'flight_duration_category',                       
            'adep_region', 
            'ades_region', 
            'same_country_flight',
            'same_region_flight',                        
            'flight_direction',
            'is_intercontinental',
            'Manufacturer',
            'Model_FAA',
            'Physical_Class_Engine',
            'FAA_Weight',
            'adep_geo_cluster',
            'ades_geo_cluster']

In [5]:
# Since we have both the challenge and submission sets locally, we will concatenate them to do data imputation (i.e. removing NaNs)
dataset = pd.concat([challenge_set_updated, submission_set_updated], axis=0)

In [6]:
# Changing display options so we can see all columns from the dataset
pd.set_option('display.max_rows', None)

# Verifying the percentage of missing values for each column in the dataset
print(dataset.isnull().mean().sort_values(ascending=False))

pd.reset_option('display.max_rows')

specific_energy_1                   0.406441
vlof_tas                            0.406441
sqrd_tas_1                          0.406441
sqrd_vlof_tas                       0.406441
vlof_groundspeed                    0.406441
tas_1                               0.406441
tas_2                               0.372550
specific_energy_2                   0.372550
sqrd_tas_2                          0.372550
tas_3                               0.359277
sqrd_tas_3                          0.359277
specific_energy_3                   0.359277
specific_energy_10                  0.356763
sqrd_tas_10                         0.356763
tas_10                              0.356763
sqrd_tas_9                          0.353705
specific_energy_9                   0.353705
tas_9                               0.353705
tas_4                               0.353320
specific_energy_4                   0.353320
sqrd_tas_4                          0.353320
tas_8                               0.351376
specific_e

In [7]:
# Dropping columns with too many NaNs
threshold = 0.4
df = dataset.dropna(thresh=int((1 - threshold) * len(dataset)), axis=1)

In [8]:
# Imputation of NaNs
columns_with_nan = dataset.isna().any()
for col in dataset.columns[columns_with_nan]:
    dataset.loc[:, col] = dataset.fillna(dataset[col].median())

In [9]:
pd.set_option('display.max_rows', None)

# Verifying if we indeed removed all NaNs
print(dataset.isnull().mean().sort_values(ascending=False))

pd.reset_option('display.max_rows')

callsign                            0.0
same_country_flight                 0.0
taxi_ratio                          0.0
flight_speed                        0.0
normalized_taxi_ratio               0.0
flight_duration_category            0.0
speed_per_distance                  0.0
cumulative_avg_speed                0.0
avg_speed_ENR                       0.0
ENR_distance_ratio                  0.0
flight_time_excl_taxi               0.0
adep_region                         0.0
ades_region                         0.0
same_region_flight                  0.0
vertical_rate_airspeed_DEP          0.0
flight_direction                    0.0
is_intercontinental                 0.0
airspeed_specific_energy_ARR        0.0
airspeed_specific_energy_DEP        0.0
airspeed_specific_energy_ENR        0.0
groundspeed_flown_distance_ARR      0.0
groundspeed_flown_distance_DEP      0.0
groundspeed_flown_distance_ENR      0.0
humidity_temperature_ARR            0.0
humidity_temperature_DEP            0.0


In [10]:
# After imputation, we recover the challenge set portion
df = dataset.iloc[0:challenge_set_updated.shape[0], :]

In [11]:
X = df.drop('tow', axis=1)
y = df.tow

In [12]:
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Create correlation matrix
# corr_matrix = X.corr(numeric_only=True).abs()

# # Select upper triangle of correlation matrix
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# # Find features with correlation greater than 0.98
# to_drop = [column for column in upper.columns if any(upper[column] > 0.98)]
# print(to_drop)

# # Plotting the correlation matrix
# plt.figure(figsize=(12, 8))
# sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
# plt.title('Correlation Matrix of Challenge Set')
# plt.show()

# # # Drop features 
# # X.drop(to_drop, axis=1, inplace=True)

In [13]:
# eliminated_features = ['groundspeed_airspeed_ratio_ENR', 'temperature_9', 'wind_distance_flown_distance_ENR', 'average_humidity_DEP_40', 'vertical_rate_bins_ARR', 
#         'groundspeed_flown_distance_ARR', 'arrival_quarter', 'offblock_year', 'arrival_year', 'offblock_to_arrival_day_diff', 'altitude_9', 'tas_1', 
#         'is_arrival_weekend', 'adep_height_6', 'sqrd_vlof_tas', 'average_airspeed_ARR_100', 'adep_height_7', 'wind_distance_ARR_100', 'altitude_4', 
#         'adep_height_1', 'groundspeed_airspeed_ratio_ARR', 'tas_8', 'specific_energy_4', 'temperature_bins_DEP', 'temperature_6', 'humidity_bins_DEP', 
#         'altitude_5', 'adep_height_5', 'sqrd_tas_8', 'sqrd_tas_7', 'specific_energy_7', 'specific_energy_1', 'adep_height_4', 'sqrd_tas_6', 'tas_2', 
#         'sqrd_tas_5', 'specific_energy_3', 'altitude_8', 'specific_energy_6', 'adep_height_8', 'vertical_rate_airspeed_ARR', 'altitude_2', 'sqrd_tas_1', 
#         'sqrd_tas_3', 'specific_energy_8', 'sqrd_tas_9', 'temperature_8', 'groundspeed_airspeed_ratio_DEP', 'sqrd_tas_4', 'altitude_6', 
#         'specific_energy_5', 'humidity_temperature_DEP', 'adep_height_2', 'altitude_7', 'adep_height_3', 'temperature_1', 'specific_energy_2', 
#         'temperature_5', 'wind_distance_flown_distance_ARR', 'arrival_month', 'temperature_4', 'groundspeed_ARR_100', 'tas_4', 'arrival_minute', 
#         'adep_height_9', 'altitude_groundspeed_ARR', 'altitude_3', 'temperature_7', 'airspeed_specific_energy_ENR', 'altitude_10', 'sqrd_tas_10', 
#         'humidity_bins_ARR', 'specific_energy_9', 'sqrd_tas_2', 'temperature_2', 'tas_10', 'average_humidity_ENR', 'offblock_quarter', 
#         'airspeed_specific_energy_DEP', 'wind_distance_flown_distance_DEP', 'tas_6', 'flown_distance_ARR_100', 'vertical_rate_airspeed_ratio_ARR', 
#         'average_humidity_ARR_100', 'specific_energy_10', 'first_adep_height', 'tas_3', 'temperature_3', 'track_variation_ARR_100', 
#         'is_offblock_rush_hour', 'average_temperature_ENR', 'is_arrival_rush_hour', 'average_altitude_ARR_100', 'specific_energy_ENR', 
#         'groundspeed_ENR', 'is_offblock_weekend', 'Num_Engines', 'temperature_bins_ARR', 'average_temperature_ARR_100', 'kpi17_time', 
#         'average_airspeed_DEP_40', 'wind_distance_ENR', 'offblock_minute', 'groundspeed_10NM', 'average_vertical_rate_ARR_100', 'vlof_tas', 
#         'humidity_temperature_ARR']

# Features eliminated by select_features() (see catboost-select.ipynb)
eliminated_features = ['track_variation_ARR_100', 'vertical_rate_bins_ARR', 'adep_height_5', 'temperature_bins_DEP', 'is_offblock_weekend', 
                       'humidity_temperature_DEP', 'humidity_bins_DEP', 'specific_energy_4', 'groundspeed_airspeed_ratio_ARR', 
                       'average_vertical_rate_ARR_100', 'flown_distance_ARR_100', 'arrival_minute', 'max_altitude_ARR_100', 
                       'wind_distance_flown_distance_ARR', 'adep_height_1', 'offblock_year', 'arrival_year', 'offblock_to_arrival_day_diff', 
                       'adep_height_7', 'specific_energy_5', 'groundspeed_flown_distance_ARR', 'altitude_5', 'groundspeed_ARR_100', 'temperature_1', 
                       'altitude_6', 'altitude_groundspeed_ARR', 'adep_height_3', 'altitude_7', 'temperature_bins_ARR', 'altitude_3', 
                       'specific_energy_2', 'humidity_bins_ARR', 'specific_energy_1', 'altitude_8', 'specific_energy_3', 'temperature_3', 
                       'sqrd_tas_6', 'adep_height_8', 'groundspeed_10NM', 'vertical_rate_airspeed_ARR', 'adep_height_2', 'offblock_quarter', 
                       'temperature_7', 'wind_distance_ARR_100', 'average_humidity_DEP_40', 'adep_height_6', 'temperature_9', 
                       'average_airspeed_ARR_100', 'adep_height_4', 'specific_energy_9', 'specific_energy_10', 'altitude_4', 'vlof_tas', 
                       'specific_energy_ENR', 'sqrd_tas_9', 'temperature_2', 'altitude_10', 'arrival_quarter', 'average_humidity_ENR', 'altitude_1', 
                       'sqrd_tas_4', 'temperature_10', 'specific_energy_7', 'wind_distance_flown_distance_ENR', 'tas_6', 'is_arrival_weekend', 
                       'specific_energy_8', 'temperature_6', 'altitude_9', 'kpi17_time', 'average_humidity_ARR_100', 'tas_8', 'first_adep_height', 
                       'tas_9', 'specific_energy_6', 'tas_1', 'vertical_rate_airspeed_ratio_ARR', 'average_altitude_ARR_100', 'temperature_5', 
                       'adep_height_9', 'tas_7', 'avg_speed_ENR', 'groundspeed_ENR', 'altitude_groundspeed_DEP', 'tas_2', 'groundspeed_DEP_40', 
                       'sqrd_vlof_tas', 'wind_distance_ENR', 'is_arrival_rush_hour', 'humidity_temperature_ARR', 'average_temperature_ARR_100', 
                       'max_altitude_ENR', 'temperature_4', 'sqrd_tas_10', 'track_variation_ENR', 'altitude_2', 'max_altitude_DEP_40', 
                       'airspeed_specific_energy_DEP', 'groundspeed_flown_distance_DEP', 'tas_3', 'groundspeed_flown_distance_ENR', 
                       'track_variation_DEP_40', 'wind_distance_DEP_40', 'arrival_day_of_week', 'tas_4', 'airspeed_specific_energy_ENR', 
                       'vertical_rate_airspeed_ratio_ENR', 'temperature_8', 'arrival_week_of_year', 'sqrd_tas_2', 'sqrd_tas_5', 'taxiout_time', 
                       'adep_height_10', 'flown_distance_kpi17', 'offblock_day_of_year', 'sqrd_tas_7', 'wind_distance_flown_distance_DEP', 
                       'average_vertical_rate_ENR', 'arrival_month', 'tas_10NM', 'flight_speed', 'altitude_groundspeed_ENR', 'sqrd_tas_8', 
                       'is_offblock_rush_hour', 'ENR_distance_ratio', 'tas_5', 'average_airspeed_DEP_40', 'bearing', 'offblock_minute', 
                       'average_altitude_DEP_40']

In [14]:
# Dropping the eliminated features
X.drop(eliminated_features, axis=1, inplace=True)

In [15]:
from catboost import CatBoostRegressor, Pool, metrics
from sklearn.model_selection import train_test_split

# Splitting the dataset into training and validation datasets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Hyperparameters found using catboost-optuna.py
params = {
    'learning_rate': 0.01, 
    'reg_lambda': 0.05357182104973179, 
    'random_strength': 20.10951792232919, 
    'depth': 9, 
    'min_data_in_leaf': 11, 
    'leaf_estimation_iterations': 12,
}

# Since some categorical features may have been dropped, we need to remove them from the list of categorical features
selected_cat_names = [x for x in cat_names if x in X.columns]

train_pool = Pool(X_train, y_train, cat_features=selected_cat_names)
val_pool = Pool(X_val, y_val, cat_features=selected_cat_names)

model = CatBoostRegressor(
    iterations=100000,
    objective=metrics.RMSE(),
    eval_metric=metrics.RMSE(),
    random_seed=42,
    verbose=False,
    task_type='GPU',
    use_best_model=True,
    od_type='Iter',
    od_wait=50,
    **params,
)

In [ ]:
# Training the model
model.fit(
    train_pool, eval_set=val_pool,
    verbose=100,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 52821.1255998	test: 52397.7535436	best: 52397.7535436 (0)	total: 147ms	remaining: 4h 5m 19s
100:	learn: 20336.2478778	test: 20184.0888408	best: 20184.0888408 (100)	total: 12.8s	remaining: 3h 31m 14s
200:	learn: 8796.5825755	test: 8749.7174979	best: 8749.7174979 (200)	total: 26.3s	remaining: 3h 37m 47s
300:	learn: 5195.1769486	test: 5188.1031951	best: 5188.1031951 (300)	total: 41.2s	remaining: 3h 47m 17s
400:	learn: 4209.3227835	test: 4216.9818638	best: 4216.9818638 (400)	total: 55.4s	remaining: 3h 49m 8s
500:	learn: 3870.9190956	test: 3888.5691654	best: 3888.5691654 (500)	total: 1m 11s	remaining: 3h 56m 8s
600:	learn: 3692.9960553	test: 3718.6122885	best: 3718.6122885 (600)	total: 1m 25s	remaining: 3h 56m 22s
700:	learn: 3565.3217155	test: 3595.3691931	best: 3595.3691931 (700)	total: 1m 39s	remaining: 3h 54m 20s
800:	learn: 3468.5242188	test: 3505.8925310	best: 3505.8925310 (800)	total: 1m 53s	remaining: 3h 54m 27s
900:	learn: 3383.0674300	test: 3426.1653626	best: 3426.165362

In [ ]:
# Saving the model trained on the train set
model.save_model('catboost_train_v%d.cbm' % version, 'cbm')

In [ ]:
# Saving the ATOWs estimated by the model for the flights on the val set (this is used to tune the ensemble model)
y_pred = model.predict(X_val)
pd.DataFrame(data={'tow': y_pred}).to_csv('catboost_val_v%d.csv' % version, index=False)

In [ ]:
# Auxiliary function to categorize the aircraft
def classify_aircraft(row):
    if row['Physical_Class_Engine'] == 'Turboprop' and row['wtc'] == 'M':
        return 'Medium Turbo Prop'
    elif row['Physical_Class_Engine'] == 'Jet' and row['wtc'] == 'M':
        return 'Medium Jet'
    elif row['Physical_Class_Engine'] == 'Jet' and row['wtc'] == 'H':
        return 'Heavy Jet'
    else:
        return None  # If no classification applies, return None

In [ ]:
# This computes separated RMSE for each airplane category. We found that most of our RMSE comes from the "Heavy Jet" aircraft.
from sklearn.metrics import root_mean_squared_error

X_val_groups = X_val.copy()

X_val_groups['Aircraft_Class'] = X_val.apply(classify_aircraft, axis=1)

X_val_groups = X_val_groups.reset_index()

indices_m_prop = X_val_groups[X_val_groups['Aircraft_Class'] == 'Medium Turbo Prop'].index
indices_m_jet = X_val_groups[X_val_groups['Aircraft_Class'] == 'Medium Jet'].index
indices_h_jet = X_val_groups[X_val_groups['Aircraft_Class'] == 'Heavy Jet'].index

print('RMSE Medium Turbo Prop:', root_mean_squared_error(y_val.iloc[indices_m_prop], y_pred[indices_m_prop]))
print('RMSE Medium Jet:', root_mean_squared_error(y_val.iloc[indices_m_jet], y_pred[indices_m_jet]))
print('RMSE Heavy Jet:', root_mean_squared_error(y_val.iloc[indices_h_jet], y_pred[indices_h_jet]))

In [ ]:
# Retrain the model using the whole dataset for maximum performance on the challenge
best_iteration = model.get_best_iteration()
best_iteration = 62049
model = CatBoostRegressor(
    iterations=best_iteration,
    objective=metrics.RMSE(),
    random_seed=42,
    verbose=False,
    task_type='GPU',
    **params,
)
model.fit(
    X, y,
    cat_features=selected_cat_names, 
    plot=True,
    verbose=100
)

In [ ]:
# Saving the model trained on the whole dataset
model.save_model('catboost_all_v%d.cbm' % version, 'cbm')

In [ ]:
# Getting feature importance
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

In [ ]:
# Plotting the feature importance

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

feature_im_df = pd.DataFrame({
    "feature": X.columns,
    "importance": feature_importances
})

feature_im_df = feature_im_df.sort_values(by="importance", ascending=False)

sns.set(rc={'text.usetex' : True, 'text.latex.preamble': '\\usepackage{libertine}'})
plt.figure(figsize=(10, 6))
plt.figure()
sns.barplot(data = feature_im_df[:20], x='importance', y='feature', palette="viridis")

plt.title("CatBoost feature importance")
plt.xlabel("Importance")
plt.ylabel("feature")
plt.tight_layout()
plt.savefig('catboost_feature_importance.pdf', bbox_inches='tight')
plt.show()

## Generating Submission

In [ ]:
# model.load_model('catboost_all_v%d.cbm' % version, 'cbm')

In [ ]:
# Recover the submission set (after imputation)
df_test = dataset.iloc[challenge_set_updated.shape[0]:, :]
df_test.head()

In [ ]:
X_test = X_test.loc[:, X_test.columns.isin(X.columns)]
X_test = df_test.drop('tow', axis=1)

In [ ]:
# Predicting the estimated ATOWs
y_pred = model.predict(X_test)
y_test = y_pred
print(y_test)

In [ ]:
# Consistency check to verify if we have the correct orders in the datasets (the output must be 1.0)
print((df_test['aircraft_type'] == submission_set_updated['aircraft_type']).mean())

In [ ]:
# Heuristically limiting the ATOW to the maximum value for the given aircraft
submission_dataset = df_test
submission_dataset['tow'] = y_pred
pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')
print((submission_dataset['tow'] > submission_dataset['MTOW_kg']).sum())
mask = (submission_dataset['tow'] > submission_dataset['MTOW_kg'])
submission_dataset.loc[mask, 'tow'] = submission_dataset.loc[mask, 'MTOW_kg']
print((submission_dataset['tow'] > submission_dataset['MTOW_kg']).sum())

In [ ]:
# Generating the submission file (after ATOW limitation)
dft0 = pd.read_csv('./data/final_submission_set.csv')
dft0['tow'] = submission_dataset['tow']
dft0[['flight_id', 'tow']].to_csv('catboost_submission_v%d.csv' % version, index=False)